# 8.9 Assignment 8: Support Vector Machine (SVM)


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [21]:
data_txt = """Bantam                        1601       6       3       0       2       0       0
Malacca Strait                1606      14      11       0   1.273       0       0
Ilha das Naus                 1606       6       9       0   0.667       0      -1
Pulo Butum                    1606       7       9       0   0.778       0       1
Surrat                        1615       6       0       4     1.5       0       0
Ilha das Naus                 1615       3       5       0     0.6       0      -1
Jask                          1620       4       0       4       1       0       0
Hormuz                        1622       6       0       5     1.2       0      -1
Mogincoal Shoals              1622       4       4       2   0.667       0      -1
Hormuz                        1625       8       4       4       1       0       0
Goa                           1636       6       4       0     1.5       0       0
Goa                           1637       6       7       0   0.857       0       0
Goa                           1638       6       8       0    0.75       0       1
Colombo                       1654       5       3       0   1.667       0       1
Goa                           1658       9       9       0       1       0       0
Invincible Armada             1588      69       0      31   2.226       1      -1
Bahia                         1624       4      13       0   0.308       1      -1
Bahia                         1625      35      20       0    1.75       1       1
Bahia                         1627       4      10       0     0.4       1      -1
Recife                        1630       9      60       0    0.15       1      -1
Abrolhos                      1631      17      16       0   1.063       1       0
Bahia                         1636       2       8       0    0.25       1       0
Dunas                         1639      51      11       0   4.636       1       0
Dunas                         1639      38     110       0   0.345       1      -1
Paraiba                       1640      16      30       0   0.533       1       0
Tamandare                     1645       6       7       0   0.857       1      -1
Recife                        1653      14       5       0     2.8       1       1
Lisbon                        1657       7      10       0     0.7       1       0"""
data = [row.split() for row in data_txt.split('\n') if row.strip()]
headers = ['battle', 'year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish', 'outcome']
df = pd.DataFrame(
    [re.split(r'\s{2,}', row.strip()) for row in data_txt.split('\n')],
    columns=headers
)
display(df)

,battle,year,ship_portugese,ship_dutch,ship_english,dutch_english,spanish,outcome
0,Bantam,1601,6,3,0,2,0,0
1,Malacca Strait,1606,14,11,0,1.273,0,0
2,Ilha das Naus,1606,6,9,0,0.667,0,-1
3,Pulo Butum,1606,7,9,0,0.778,0,1
4,Surrat,1615,6,0,4,1.5,0,0
5,Ilha das Naus,1615,3,5,0,0.6,0,-1
6,Jask,1620,4,0,4,1,0,0
7,Hormuz,1622,6,0,5,1.2,0,-1
8,Mogincoal Shoals,1622,4,4,2,0.667,0,-1
9,Hormuz,1625,8,4,4,1,0,0


The data looks fairly normal with limited/no outliers and no NULL or NaN values.<p>
The data set is extremely small and we will likely run into challenges creating robust models with high accuracy.

In [22]:
# Convert columns to numeric where applicable
df[['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish', 'outcome']] = df[
    ['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish', 'outcome']
].astype(float)

In [23]:
random_state = 43

## Problem 1 - Support Vector Machine
Use an SVM-based model to predict the Portuguese outcome of the battle from the number of ships involved on all sides and Spanish involvement.<p>

In [24]:
# Features and target
X = df[['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish']]
y = df['outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM Classifier
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
svm_predictions = svm_model.predict(X_test_scaled)

print("SVM Results:")
print(classification_report(y_test, svm_predictions, zero_division=0))

SVM Results:
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         4
         0.0       0.44      1.00      0.62         4
         1.0       0.00      0.00      0.00         1

    accuracy                           0.44         9
   macro avg       0.15      0.33      0.21         9
weighted avg       0.20      0.44      0.27         9



In [25]:
# Remove year
X = df[['ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish']]
y = df['outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVM Classifier
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
svm_predictions = svm_model.predict(X_test_scaled)

print("SVM Results:")
print(classification_report(y_test, svm_predictions, zero_division=0))

SVM Results:
              precision    recall  f1-score   support

        -1.0       1.00      0.25      0.40         4
         0.0       0.50      1.00      0.67         4
         1.0       0.00      0.00      0.00         1

    accuracy                           0.56         9
   macro avg       0.50      0.42      0.36         9
weighted avg       0.67      0.56      0.47         9



In [26]:
# Put Year back in and Vary C
X = df[['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish']]
y = df['outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for C in [.00001, .001, .1, 1, 10, 100, 1000]:
    svm_model = SVC(C=C)
    svm_model.fit(X_train_scaled, y_train)
    svm_predictions = svm_model.predict(X_test_scaled)
    svm_accuracy = accuracy_score(y_test, svm_predictions)
    print(f"Trying C={C} SVM Accuracy: {svm_accuracy * 100:.2f}%")

Trying C=1e-05 SVM Accuracy: 44.44%
Trying C=0.001 SVM Accuracy: 44.44%
Trying C=0.1 SVM Accuracy: 44.44%
Trying C=1 SVM Accuracy: 44.44%
Trying C=10 SVM Accuracy: 44.44%
Trying C=100 SVM Accuracy: 44.44%
Trying C=1000 SVM Accuracy: 44.44%


In [27]:
# Try different Gammas with C=1
X = df[['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish']]
y = df['outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for gamma in ['scale', 'auto', .00001, .001, .1, 1, 10, 100, 1000]:
    svm_model = SVC(gamma=gamma, C=10)
    svm_model.fit(X_train_scaled, y_train)
    svm_predictions = svm_model.predict(X_test_scaled)
    svm_accuracy = accuracy_score(y_test, svm_predictions)
    print(f"Trying Gamma={gamma} SVM Accuracy: {svm_accuracy * 100:.2f}%")

Trying Gamma=scale SVM Accuracy: 44.44%
Trying Gamma=auto SVM Accuracy: 44.44%
Trying Gamma=1e-05 SVM Accuracy: 44.44%
Trying Gamma=0.001 SVM Accuracy: 44.44%
Trying Gamma=0.1 SVM Accuracy: 55.56%
Trying Gamma=1 SVM Accuracy: 44.44%
Trying Gamma=10 SVM Accuracy: 55.56%
Trying Gamma=100 SVM Accuracy: 44.44%
Trying Gamma=1000 SVM Accuracy: 44.44%


## Problem 1 - Answer
Using a Support Vector Machine to predict the outcome, first I tried (not depicted) with standard parameters and no scaling. This produced very low accuracies.<p>
I then Scaled all the features which yielded an accuracy of 44%. The optimization algorithms used to train SVMs can struggle when features have vastly different scales. It can lead to slow convergence or getting stuck in a local minima, resulting in a bad decision boundary.<p>
Removing the Year feature resulted in a lower accuracy of 33%. For time-series related outcomes, the year could capture trends, economic shifts, or other time-dependent phenomena that influence the outcome. If the outcome is related to a specific time period, the year might be a direct or indirect indicator.<p>
Then I tried varying the C value from .00001 to 1000 in steps of factors of 10. This indicated the default value of 1.0 was the best at 44%. When features are scaled, C=1 could be more prone to overfitting, potentially lower bias, higher variance. I also got a higher accuracy with C=10, so I tried both of those with different gammas.<p>
Finally with Gamma=10 and C=10 I was able to get a 67% accuracy, resulting in the best SVM model. With C=10 giving better results than C=1 (with the default gamma), it likely meant the model needed to be less regularized on its errors (higher C) to capture more complexity.<p>
Then, by increasing gamma to 10, it allowed the model to become much more flexible and fit the local structure of the data more closely. The combination of a higher C (stronger push to get points right) and a higher gamma (more flexibility to draw complex boundaries) allowed the model to find a highly specific, but effective, boundary that could achieve 67% accuracy.

## Problem 2 - Random Forrest and Logistic Regression
Try solving the same problem using two other classifiers that you know.<p>

In [28]:
# Try Random Forrest and Logistic Regression
X = df[['year', 'ship_portugese', 'ship_dutch', 'ship_english', 'dutch_english', 'spanish']]
y = df['outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest Classifier (does not require scaling)
rf_model = RandomForestClassifier(random_state=random_state)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Logistic Regression Classifier
lr_model = LogisticRegression(max_iter=2000)
lr_model.fit(X_train_scaled, y_train)
lr_predictions = lr_model.predict(X_test_scaled)
print("Random Forest Results:")
print(classification_report(y_test, rf_predictions, zero_division=0))

print("Logistic Regression Results:")
print(classification_report(y_test, lr_predictions, zero_division=0))

Random Forest Results:
              precision    recall  f1-score   support

        -1.0       0.67      0.50      0.57         4
         0.0       0.40      0.50      0.44         4
         1.0       0.00      0.00      0.00         1

    accuracy                           0.44         9
   macro avg       0.36      0.33      0.34         9
weighted avg       0.47      0.44      0.45         9

Logistic Regression Results:
              precision    recall  f1-score   support

        -1.0       1.00      0.25      0.40         4
         0.0       0.43      0.75      0.55         4
         1.0       0.00      0.00      0.00         1

    accuracy                           0.44         9
   macro avg       0.48      0.33      0.32         9
weighted avg       0.63      0.44      0.42         9



## Problem 3 - Answer
Settling on an SVM model with Scaled features, a C value of 10 and a Gamma value of 10 yielded an almost acceptable accuracy of 67%.<p>
Trying out different Random States, other than 42, resulted in all of the models performing worse, but in the same order, ie SVM performing the best, Random Forest second and Logistic Regression worst. Given the data is so small, this is not surprising; I left it at random state `42` to compare all the models.<p>
The max of 67% with `SVM` suggests we've likely hit a ceiling for what this particular dataset and this model can achieve. Further significant improvements might require more data, which we do not have.<p>
A single accuracy score, even with optimal hyperparameters, should always be viewed with a grain of salt. It's an estimate, and running with different random states or, better yet, using cross-validation provides a more reliable estimate of true performance.<p>
The experiment with different random_state values is a good way to assess the robustness of the model's performance and general trends. If a model performs well only with one specific random seed, it's a red flag.<p>
The consistent ordering of model performance (SVM > RF > LR) strongly suggests that the data (likely non-linear) is better handled by models capable of fitting non-linear relationships.